# Cloud Native Exploration of SHYFEM model on EGI
* Read model output (NetCDF files) directly from S3-compatible MinIO object storage on EGI
* Explore regridded output
* Explore original triangular mesh output

In [ ]:
import xarray as xr
import hvplot.xarray
import intake
import os

In [ ]:
from dotenv import load_dotenv
_ = load_dotenv(f'{os.environ['HOME']}/dotenv/school_2025.env')

In [ ]:
url = 's3://rsignell4-protocoast/catalogs/protocoast_intake.yml'

In [ ]:
so = dict(anon=True, endpoint_url='https://pangeo-eosc-minioapi.vm.fedcloud.eu')

In [ ]:
cat = intake.open_catalog(url, storage_options=so)

In [ ]:
list(cat)

In [ ]:
%%time
ds = cat['Taranto-SHYFEM'].to_dask()

In [ ]:
ds

## SHYFEM output on native triangular mesh

In [ ]:
vname = 'water_level'

In [ ]:
%%time
var = ds[vname].isel(time=-1).load()

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
v = np.vstack((ds['longitude'], ds['latitude'], var)).T
verts = pd.DataFrame(v, columns=['x','y','var'])

In [ ]:
import numpy as np
import datashader as dshade
import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs
import hvplot.xarray
import holoviews.operation.datashader as dshade

dshade.datashade.precompute = True
hv.extension('bokeh')

In [ ]:
points = gv.operation.project_points(gv.Points(verts, vdims=['var']))

In [ ]:
tris = pd.DataFrame(ds['element_index'].values.astype('int')-1, columns=['v0','v1','v2'])

In [ ]:
tiles = gv.tile_sources.OSM

In [ ]:
trimesh = gv.TriMesh((tris, points), label=vname)
trimesh.opts(tools=['hover'])
mesh = dshade.rasterize(trimesh).opts(cmap='turbo', colorbar=True, width=600, height=400)

In [ ]:
tiles * mesh